# Restricted Boltzmann Machine
## Curso 2 Udemy

## Actualizar a Python 3.10

In [1]:
!sudo apt-get install python3.10
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10
!sudo update-alternatives --config python3
!sudo apt install python3-pip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10-minimal
Suggested packages:
  python3.10-venv binfmt-support
The following NEW packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10 python3.10-minimal
0 upgraded, 4 newly installed, 0 to remove and 4 not upgraded.
Need to get 5,098 kB of archives.
After this operation, 19.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.10-minimal amd64 3.10.8-1+bionic1 [823 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.10-minimal amd64 3.10.8-1+bionic1 [1,970 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bioni

In [2]:
!python3 --version

Python 3.10.8


In [3]:
!nvidia-smi 

Sat Nov  5 00:02:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Librerias

In [4]:
import tensorflow as tf
tf.keras.backend.clear_session()
print(tf.__version__)

2.9.2


In [5]:
import torch
print(torch.__version__)

1.12.1+cu113


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
import sklearn 
# import theano

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
cd /content/drive/My\ Drive/AI\ Courses\ Udemy/Deep/Course\ 2/BM

/content/drive/My Drive/AI Courses Udemy/Deep/Course 2/BM


In [15]:
!pwd
!ls

/content/drive/My Drive/AI Courses Udemy/Deep/Course 2/BM
boltzmann_machine.ipynb  ml-100k  ml-1m


## Import Data Set of Movies

In [23]:
movies_dataset = pd.read_csv('ml-1m/movies.dat', 
                     sep='::', 
                     header=None, # 'infer': inferir los nombres de las columnas.
                     engine='python',
                     encoding='latin-1' # UTF-8
)
movies_dataset

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [24]:
users_dataset = pd.read_csv('ml-1m/users.dat', 
                     sep='::', 
                     header=None, # 'infer': inferir los nombres de las columnas.
                     engine='python',
                     encoding='latin-1' # UTF-8
)
users_dataset

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [25]:
ratings_dataset = pd.read_csv('ml-1m/ratings.dat', 
                     sep='::', 
                     header=None, # 'infer': inferir los nombres de las columnas.
                     engine='python',
                     encoding='latin-1' # UTF-8
)
ratings_dataset

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


## Importar Data Set de Entrenamiento y Testing.

In [66]:
# sep='\t' : los datos estan separados por tabulador.
training_set = pd.read_csv('ml-100k/u1.base', sep='\t', header=None)

# pytorch no utiliza DataFrame, sino Arrays.
training_set = pd.array(training_set, dtype='int')
print(training_set[0])
print(len(training_set))

<PandasArray>
[1, 1, 5, 874965758]
Length: 4, dtype: int64
80000


In [67]:
# sep='\t' : los datos estan separados por tabulador.
testing_set = pd.read_csv('ml-100k/u1.test', sep='\t', header=None)

# pytorch no utiliza DataFrame, sino Arrays.
testing_set = pd.array(testing_set, dtype='int')
print(testing_set[0])
print(len(testing_set))

<PandasArray>
[1, 6, 5, 887431973]
Length: 4, dtype: int64
20000


### Obtener el maximo numero (indice) de users y movies.
Cantidad de users y movies.

In [68]:
max_user = int(max(max(training_set[:, 0]), max(testing_set[:, 0])))
max_movie = int(max(max(training_set[:, 1]), max(testing_set[:, 1])))

print("User: ", max_user, "\nMovie: ", max_movie)

User:  943 
Movie:  1682


### Construir/Convertir un array con filas la cantidad de usuarios y columnas la cantidad de peliculas.
Cada pelicula tendra la valoracion del usuario correspondiente.

In [69]:
# Construir Array Bi-Dimensional de usuarios valorando cada una de todas las peliculas.
def convert(data):
  build_matrix = []
  for id_user in range(1, max_user+1):
    # Obtener todos los id_movie de un solo usuario (user_id).
    id_movies = data[:, 1][data[:, 0] == id_user]

    # Obtener todos los id_ratings de un solo usuario (user_id).
    id_ratings = data[:, 2][data[:, 0] == id_user]

    # Array de 0.
    ratings = np.zeros(max_movie)

    # Donde el valor del campo del Array de 0 sea igual al 'id_ratings'
    # --> modificar el valor del cambo por el 'id_ratings'
    ratings[id_movies-1] = id_ratings

    build_matrix.append(list(ratings))
  
  return build_matrix

In [70]:
print(training_set[0])
# Construir Array Bi_Dimensional de usuarios con todas las valoraciones de peliculas de cada usuario.
training_set_convert = convert(training_set)
print(training_set_convert[0])
print(len(training_set_convert))
print(len(training_set_convert[0]))

<PandasArray>
[1, 1, 5, 874965758]
Length: 4, dtype: int64
[5.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0, 2.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 4.0, 5.0, 0.0, 1.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 1.0, 3.0, 0.0, 5.0, 0.0, 2.0, 1.0, 0.0, 2.0, 3.0, 0.0, 3.0, 2.0, 5.0, 4.0, 0.0, 5.0, 4.0, 0.0, 5.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 5.0, 2.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0, 0.0, 5.0, 1.0, 5.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 4.0, 5.0, 0.0, 2.0, 5.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 0.0, 4.0, 3.0, 5.0, 1.0, 3.0, 0.0, 3.0, 2.0, 0.0, 4.0, 0.0, 4.0, 3.0, 0.0, 2.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 5.0, 2.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 5.0, 3.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 

In [71]:
print(testing_set[0])

testing_set_convert = convert(testing_set)
print(testing_set_convert[0])
print(len(testing_set_convert))
print(len(testing_set_convert[0]))

<PandasArray>
[1, 6, 5, 887431973]
Length: 4, dtype: int64
[0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 4.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 4.0, 0.0, 3.0, 3.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 4.0, 3.0, 0.0, 5.0, 4.0, 0.0, 3.0, 0.0, 3.0, 3.0, 0.0, 4.0, 3.0, 1.0, 0.0, 4.0, 0.0, 1.0, 0.0, 4.0, 5.0, 5.0, 0.0, 4.0, 3.0, 5.0, 0.0, 0.0, 0.0, 4.0, 5.0, 3.0, 0.0, 0.0, 0.0, 5.0, 3.0, 4.0, 0.0, 5.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 1.0, 5.0, 5.0, 0.0, 0.0, 3.0, 3.0, 0.0, 1.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 5.0, 3.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 5.0, 4.0, 0.0, 0.0, 5.0, 2.0, 0.0, 4.0, 0.0, 3.0, 4.0, 4.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 4.0, 4.0, 4.0, 0.0, 3.0, 

## Convertir los datos a Tensores Torch.

In [73]:
training_set_tensor = torch.FloatTensor(training_set_convert)
testing_set_tensor = torch.FloatTensor(testing_set_convert)

print(training_set_tensor)
print(testing_set_tensor)

tensor([[5., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


### Convertir a Binarios los valores de los vectores.

In [74]:
training_set_tensor[training_set_tensor == 0] = -1
training_set_tensor[training_set_tensor == 1] = 0
training_set_tensor[training_set_tensor == 2] = 0
training_set_tensor[training_set_tensor >= 3] = 1

testing_set_tensor[testing_set_tensor == 0] = -1
testing_set_tensor[testing_set_tensor == 1] = 0
testing_set_tensor[testing_set_tensor == 2] = 0
testing_set_tensor[testing_set_tensor >= 3] = 1

## Crear la ANN : Restricted Boltzmann Machine.
### Modelo Probabilistico Grafico.

In [157]:
class RBM:
  def __init__(self, nv, nh):
    # Inicializar un Tensor de (nh * nv).
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)

  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)

  def train(self, v0, vk, ph0, phk):
    """ Actualizacion de los pesos de la RBM """
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)

  def __str__(self):
    print("Soy un RBM.")

  def __delete__(self):
    print("Deleteee...")


In [158]:
# nv = nodos visibles : cantidad de variables de entrada (Input Features). 
nv = len(training_set_tensor[0])

# nh = nodos ocultos : cantodad de caracteristicas que se van a instanciar para
# --> cmparar entre las peliculas.
nh = 100

batch_size = 128

""" Crear objeto Boltzmann Machine. """
rbm_obj = RBM(nv, nh)
print(rbm_obj.__str__)

<bound method RBM.__str__ of <__main__.RBM object at 0x7fa47f387110>>


## Entrenar la Restricted Boltzmann Machine ANN.

In [159]:
nb_epoch = 10

for epoch in range(1, nb_epoch+1):
  training_loss = 0
  s = 0.

  for id_user in range(0, max_user - batch_size, batch_size):
    vk = training_set_tensor[id_user:id_user + batch_size]
    v0 = training_set_tensor[id_user:id_user + batch_size]
    ph0, _ = rbm_obj.sample_h(v0)

    for k in range(10):
      _, hk = rbm_obj.sample_h(vk)
      _, vk = rbm_obj.sample_v(hk)
      vk[v0 < 0] = v0[v0 < 0]
    
    phk, _ = rbm_obj.sample_h(vk)
    rbm_obj.train(v0, vk, ph0, phk)

    training_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))

    # RMSE es un metodo para evaluar la RBM. Distancia promedio es otro metodo.
    # training_loss += np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2))
    s += 1.
    
  """ Visualizacion del progreso de entrenamiento """
  print("Epoch: " + str(epoch))
  print("Loss: " + str(training_loss / s))

Epoch: 1
Loss: tensor(0.3608)
Epoch: 2
Loss: tensor(0.2565)
Epoch: 3
Loss: tensor(0.2376)
Epoch: 4
Loss: tensor(0.2487)
Epoch: 5
Loss: tensor(0.2479)
Epoch: 6
Loss: tensor(0.2502)
Epoch: 7
Loss: tensor(0.2456)
Epoch: 8
Loss: tensor(0.2477)
Epoch: 9
Loss: tensor(0.2460)
Epoch: 10
Loss: tensor(0.2446)


## Testear la RBM.

In [160]:
testing_loss = 0
s = 0.

for id_user in range(max_user):
  # Hay que mantener el conjunto de entramiento por que es el esquema que conoce la RBM.
  v = training_set_tensor[id_user:id_user + 1]
  vt = testing_set_tensor[id_user:id_user + 1]

  if len(vt[vt >= 0]):
    _, h = rbm_obj.sample_h(v)
    _, v = rbm_obj.sample_v(h)

    testing_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))

    # RMSE es un metodo para evaluar la RBM. Distancia promedio es otro metodo.
    # rmse = testing_loss + np.sqrt(torch.mean((vt[vt>=0] - v[vt>=0])**2)) 
    s += 1.
  
    """ Visualizacion del progreso de entrenamiento """
    print("Testing Loss: " + str(testing_loss / s))

Testing Loss: tensor(0.2920)
Testing Loss: tensor(0.2369)
Testing Loss: tensor(0.2990)
Testing Loss: tensor(0.2742)
Testing Loss: tensor(0.2908)
Testing Loss: tensor(0.2753)
Testing Loss: tensor(0.2638)
Testing Loss: tensor(0.2524)
Testing Loss: tensor(0.2355)
Testing Loss: tensor(0.2319)
Testing Loss: tensor(0.2383)
Testing Loss: tensor(0.2345)
Testing Loss: tensor(0.2422)
Testing Loss: tensor(0.2387)
Testing Loss: tensor(0.2501)
Testing Loss: tensor(0.2467)
Testing Loss: tensor(0.2518)
Testing Loss: tensor(0.2482)
Testing Loss: tensor(0.2457)
Testing Loss: tensor(0.2538)
Testing Loss: tensor(0.2633)
Testing Loss: tensor(0.2607)
Testing Loss: tensor(0.2591)
Testing Loss: tensor(0.2560)
Testing Loss: tensor(0.2501)
Testing Loss: tensor(0.2555)
Testing Loss: tensor(0.2583)
Testing Loss: tensor(0.2589)
Testing Loss: tensor(0.2642)
Testing Loss: tensor(0.2684)
Testing Loss: tensor(0.2696)
Testing Loss: tensor(0.2727)
Testing Loss: tensor(0.2675)
Testing Loss: tensor(0.2684)
Testing Loss: 